In [5]:
import torch
import numpy as np

from torch.nn import Parameter, Linear, BatchNorm1d, ReLU, LeakyReLU, Linear, Dropout, CrossEntropyLoss, BCEWithLogitsLoss
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAccuracy, BinaryAUROC
from torchmetrics.regression import R2Score, MeanSquaredError, MeanAbsoluteError
from torchmetrics.classification import Accuracy, AUROC

from molsetrep.utils.torch_trainer import TorchTrainer
from molsetrep.utils.multiset_torch_trainer import MultisetTorchTrainer
from molsetrep.utils.datasets import molnet_loader
from molsetrep.utils.converters import molnet_to_pyg
from molsetrep.utils.root_mean_squared_error import RootMeanSquaredError
from molsetrep.utils.imbalanced_sampler import ImbalancedSampler
from molsetrep.encoders import TripleSetEncoder
from molsetrep.models import FocalLoss

from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

import matplotlib.pyplot as plt

import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import wandb
from wandb import finish as wandb_finish


## Setup

### Lightning Module

In [10]:
class TripleSetRegressor(pl.LightningModule):
    def __init__(
        self,
        n_hidden_sets,
        n_hidden_sets_2,
        n_hidden_sets_3,
        n_elements,
        n_elements_2,
        n_elements_3,
        d,
        d_2,
        d_3,
        learning_rate = 0.001,
        fc_1_units = 32,
    ):
        super().__init__()
        self.save_hyperparameters()

        self.n_hidden_sets = n_hidden_sets
        self.n_elements = n_elements

        self.n_hidden_sets_2 = n_hidden_sets_2
        self.n_elements_2 = n_elements_2

        self.n_hidden_sets_3 = n_hidden_sets_3
        self.n_elements_3 = n_elements_3

        self.learning_rate = learning_rate

        self.fc_1_units = fc_1_units

        self.Wc = Parameter(torch.FloatTensor(d, n_hidden_sets * n_elements))
        self.Wc_2 = Parameter(torch.FloatTensor(d_2, n_hidden_sets_2 * n_elements_2))
        self.Wc_3 = Parameter(torch.FloatTensor(d_3, n_hidden_sets_3 * n_elements_3))

        self.bn = BatchNorm1d(n_hidden_sets)
        self.bn_2 = BatchNorm1d(n_hidden_sets_2)
        self.bn_3 = BatchNorm1d(n_hidden_sets_3)

        self.fc1 = Linear(n_hidden_sets, self.fc_1_units)
        self.fc1_2 = Linear(n_hidden_sets_2, self.fc_1_units)
        self.fc1_3 = Linear(n_hidden_sets_3, self.fc_1_units)

        # MLP
        self.fc2 = Linear(self.fc_1_units * 3, 32)
        self.bn_mlp = BatchNorm1d(32)
        self.fc3 = Linear(32, 16)
        self.fc4 = Linear(16, 1)

        # Init weights
        self.Wc.data.uniform_(-1, 1)
        self.Wc_2.data.uniform_(-1, 1)
        self.Wc_3.data.uniform_(-1, 1)

        # Metrics
        self.train_r2 = R2Score()
        self.train_rmse = MeanSquaredError(squared=False)
        self.val_r2 = R2Score()
        self.val_rmse = MeanSquaredError(squared=False)
        self.test_r2 = R2Score()
        self.test_rmse = MeanSquaredError(squared=False)


    def forward(self, X, X2, X3):
        # First sets (e.g. atoms)
        t = torch.matmul(X, self.Wc)

        t = torch.relu(t)
        t = t.view(t.size()[0], t.size()[1], self.n_elements, self.n_hidden_sets)
        t, _ = torch.max(t, dim=2)
        t = torch.sum(t, dim=1)
        t = self.bn(t)

        t = self.fc1(t)
        t = torch.relu(t)

        # Second sets (e.g. bonds)
        t_2 = torch.matmul(X2, self.Wc_2)
        t_2 = torch.relu(t_2)
        t_2 = t_2.view(
            t_2.size()[0], t_2.size()[1], self.n_elements_2, self.n_hidden_sets_2
        )
        t_2, _ = torch.max(t_2, dim=2)
        t_2 = torch.sum(t_2, dim=1)
        t_2 = self.bn_2(t_2)
        t_2 = self.fc1_2(t_2)
        t_2 = torch.relu(t_2)

        # Third sets
        t_3 = torch.matmul(X3, self.Wc_3)
        t_3 = torch.relu(t_3)
        t_3 = t_3.view(
            t_3.size()[0], t_3.size()[1], self.n_elements_3, self.n_hidden_sets_3
        )
        t_3, _ = torch.max(t_3, dim=2)
        t_3 = torch.sum(t_3, dim=1)
        t_3 = self.bn_3(t_3)
        t_3 = self.fc1_3(t_3)
        t_3 = torch.relu(t_3)

        # Concat, mlp, and softmax
        out = self.fc2(torch.cat((t, t_2, t_3), 1))

        out = self.bn_mlp(out)
        out = torch.relu(out)
        out = self.fc3(out)
        out = torch.relu(out)
        out = self.fc4(out)

        return out.squeeze(1)

    def training_step(self, batch, batch_idx):
        x, x2, x3, y = batch

        out = self(x, x2, x3)
        loss = F.mse_loss(out, y)

        # Metrics
        self.train_r2(out, y)
        self.train_rmse(out, y)

        self.log("train/loss", loss, on_step=False, on_epoch=True)
        self.log("train/r2", self.train_r2, on_step=False, on_epoch=True)
        self.log("train/rmse", self.train_rmse, on_step=False, on_epoch=True)

        return loss

    def validation_step(self, val_batch, batch_idx):
        x, x2, x3, y = val_batch

        out = self.forward(x, x2, x3)
        loss = F.mse_loss(out, y)

        # Metrics
        self.val_r2(out, y)
        self.val_rmse(out, y)

        self.log("val/loss", loss, on_step=False, on_epoch=True)
        self.log("val/r2", self.val_r2, on_step=False, on_epoch=True)
        self.log("val/rmse", self.val_rmse, on_step=False, on_epoch=True)

    def test_step(self, val_batch, batch_idx):
        x, x2, x3, y = val_batch

        out = self.forward(x, x2, x3)
        loss = F.mse_loss(out, y)

        # Metrics
        self.test_r2(out, y)
        self.test_rmse(out, y)

        self.log("test/loss", loss, on_step=False, on_epoch=True)
        self.log("test/r2", self.test_r2, on_step=False, on_epoch=True)
        self.log("test/rmse", self.test_rmse, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

## Train

### Load Data & Fit

In [11]:
data_set_name = "lipo"

train, valid, test, tasks = molnet_loader(data_set_name, splitter="random")
enc = TripleSetEncoder()

for task in range(len(tasks)):
    train_dataset = enc.encode(train.ids, [y[task] for y in train.y], label_dtype=torch.float)
    valid_dataset = enc.encode(valid.ids, [y[task] for y in valid.y], label_dtype=torch.float)
    test_dataset = enc.encode(test.ids, [y[task] for y in test.y], label_dtype=torch.float)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=8, drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8, drop_last=True)

    d = len(train_dataset[0][0][0])
    d2 = len(train_dataset[0][1][0])
    d3 = len(train_dataset[0][2][0])

    for _ in range(4):
        # Make sure no run is ongoing
        wandb_finish()
        
        # Setup wandb logging
        wandb_logger = wandb.WandbLogger(project=f"MolRepSet-triple-{data_set_name}")
        wandb_logger.experiment.config["task"] = tasks[task]

        # Define callback for which callpoint to load for testing
        checkpoint_callback = ModelCheckpoint(monitor="val/rmse", mode="min")

        trainer = pl.Trainer(callbacks=[checkpoint_callback], max_epochs=200, log_every_n_steps=1, logger=wandb_logger)

        model = TripleSetRegressor(64, 64, 16, 32, 32, 8, d, d2, d3, learning_rate=0.001)
        wandb_logger.watch(model, log="all")

        trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader)
        trainer.test(ckpt_path="best", dataloaders=test_loader)

        wandb_logger.finalize("success")
        wandb_finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type             | Params
-------------------------------------------------
0  | bn         | BatchNorm1d      | 128   
1  | bn_2       | BatchNorm1d      | 128   
2  | bn_3       | BatchNorm1d      | 32    
3  | fc1        | Linear           | 2.1 K 
4  | fc1_2      | Linear           | 2.1 K 
5  | fc1_3      | Linear           | 544   
6  | fc2        | Linear           | 3.1 K 
7  | bn_mlp     | BatchNorm1d      | 64    
8  | fc3        | Linear           | 528   
9  | fc4        | Linear           | 17    
10 | train_r2   | R2Score          | 0     
11 | train_rmse | MeanSquaredError | 0     
12 | val_r2     | R2Score          | 0     
13 | val_rmse   | MeanSquaredError | 0     
14 | test_r2    | R2Score  

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.
Restoring states from the checkpoint path at ./MolRepSet-triple-lipo/omzlf28e/checkpoints/epoch=146-step=7644.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at ./MolRepSet-triple-lipo/omzlf28e/checkpoints/epoch=146-step=7644.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/loss         │    0.4656083881855011     │
│          test/r2          │    0.5213297605514526     │
│         test/rmse         │    0.6823550462722778     │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test/loss,▁
test/r2,▁
test/rmse,▁
train/loss,█▆▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/r2,▁▃▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████
train/rmse,█▆▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/loss,█▅▄▂▃▂▄▂▂▂▁▂▂▂▁▂▂▂▄▂▃▁▂▂▁▁▁▁▂▁▃▂▃▁▄▅▁▅▁▁
val/r2,▁▄▅▇▆▇▅▇▇▇█▇▇▇█▇▇▇▅▇▆█▇▇████▇█▆▇▆█▅▄█▄██
val/rmse,█▅▄▂▃▂▅▂▂▂▁▂▂▂▁▂▂▂▄▂▃▁▂▂▁▁▂▁▂▁▃▂▄▁▅▅▁▅▁▁
